In [ ]:
import pandas as pd
import numpy as np
import json

from sklearn.model_selection import train_test_split

# Load data

In [ ]:
raw = pd.read_csv("../data/myauto-raw.csv")

In [ ]:
with open("../data/id_to_name.json") as file:
    id_to_name = json.load(file)

# Select columns and drop duplicates

In [ ]:
maybe = ['photo_ver', 'stickers', 'prom_color', 'video_url', 'checked', 'words_checked', 'auction', 'auction_has_key', 'is_auction', 'car_desc', 'lang_type_id', 'parent_loc_id', 'car_run_dim', 'car_run', 'leather']
columns_to_use = ['paid_add', 'prod_year', 'man_id', 'price', 'price_value', 'fuel_type_id', 'gear_type_id', 'drive_type_id', 'door_type_id', 'color_id', 'saloon_color_id', 'cylinders', 'car_run_km', 'engine_volume', 'airbags', 'abs', 'esd', 'el_windows', 'conditioner', 'disks', 'nav_system', 'central_lock', 'hatch', 'right_wheel', 'alarm', 'board_comp', 'hydraulics', 'chair_warming', 'climat_control', 'obstacle_indicator', 'customs_passed', 'model_id', 'location_id', 'tech_inspection', 'has_turbo', 'for_rent', 'rent_daily', 'rent_purchase', 'rent_insured', 'rent_driver', 'currency_id', 'category_id', 'vin', 'back_camera', 'order_date', 'hp', 'el_starter', 'start_stop', 'trunk', 'windshield', 'inspected_in_greenway', 'is_payd', 'condition_type_id', 'saloon_material_id', 'predicted_price', 'has_catalyst', 'views', 'has_predicted_price', 'comfort_features', 'pred_min_price', 'pred_max_price']

In [ ]:
raw = raw[sorted(columns_to_use)]
raw.rename(columns = {
    'price_value': 'price_in_gel',
    'price': 'price_original',
    'climat_control': 'climate_control'
}, inplace = True)

In [ ]:
raw.drop_duplicates(inplace = True)
raw.reset_index(inplace = True, drop = True)

In [ ]:
pd.set_option('display.max_columns', 200)
raw.tail()

# Map feature IDs to names

In [ ]:
id_to_name

In [ ]:
def parse(dict_of_lists, pref = ""):
    if type(dict_of_lists) is dict:
        return pd.concat([pd.DataFrame(elem).add_prefix(pref + key.lower() + "::") if type(elem) is list else parse(elem, pref + key.lower() + "::") for key, elem in dict_of_lists.items()], axis = 1)
    else:
        return pd.DataFrame()

In [ ]:
df = parse(id_to_name)

In [ ]:
def map_id_to_name(raw_id, raw_name, idd, name):
    map_it = df[[idd, name]].dropna()
    map_it[idd] = map_it[idd].astype(int)
    map_it = map_it.set_index(idd)
    
    raw[raw_name] = raw[raw_id].astype(int).map(map_it.to_dict()[name])

In [ ]:
pd.set_option('display.max_rows', 200)
df['comfortfeatures::items::title'].dropna()

In [ ]:
feature_id = ['category_id', 'color_id', 'condition_type_id', 'currency_id', 'door_type_id', 'drive_type_id',\
             'fuel_type_id', 'gear_type_id', 'location_id', 'man_id', 'model_id', 'saloon_color_id', 'saloon_material_id']

In [ ]:
map_id_to_name('category_id', 'body_type', 'categories::category_id', 'categories::title')
raw['body_type'] = raw['body_type'].replace('Jeep', 'suv')

In [ ]:
map_id_to_name('color_id', 'color', 'colors::color_id', 'colors::color_name')
map_id_to_name('color_id', 'color_rgb', 'colors::color_id', 'colors::color_code')

In [ ]:
map_id_to_name('currency_id', 'currency', 'currencies::currencyID', 'currencies::title')

In [ ]:
map_id_to_name('door_type_id', 'door_type', 'doortypes::door_type_id', 'doortypes::door_type_name')

In [ ]:
map_id_to_name('drive_type_id', 'drive_type', 'drivetypes::drive_type_id', 'drivetypes::drive_type_name')

In [ ]:
map_id_to_name('fuel_type_id', 'fuel_type', 'fueltypes::items::fuel_type_id', 'fueltypes::items::title')

In [ ]:
map_id_to_name('gear_type_id', 'gear_type', 'geartypes::items::gear_type_id', 'geartypes::items::title')

In [ ]:
map_id_to_name('location_id', 'location', 'locations::items::location_id', 'locations::items::title')

In [ ]:
map_id_to_name('man_id', 'manufacturer', 'mans::man_id', 'mans::man_name')

In [ ]:
map_id_to_name('model_id', 'model', 'models::model_id', 'models::model_name')
map_id_to_name('model_id', 'model_group', 'models::model_id', 'models::model_group')

In [ ]:
map_id_to_name('saloon_color_id', 'saloon_color', 'salooncolors::saloon_color_id', 'salooncolors::color_name')
map_id_to_name('saloon_color_id', 'saloon_color_rgb', 'salooncolors::saloon_color_id', 'salooncolors::color_code')

In [ ]:
map_id_to_name('saloon_material_id', 'saloon_material', 'salooninteriormaterial::id', 'salooninteriormaterial::title')

In [ ]:
raw.drop(feature_id, inplace = True, axis = 1)

In [ ]:
raw.head()

## Translate comfort features using Google Translate and then map
<b> In general, test data should not be used to generate variables like below </b>, as some of the tags in the test set may not appear in the train set - we are effectively leaking the test data. 

I will encode everything so I don't have to share the raw sources (and for your convenience), and filter out the tags that are always False in the trainset when train-test splitting the data. 

In [ ]:
feature_id_list = ['comfort_features']

In [ ]:
from googletrans import Translator
import time
#catcomfortfeatures::id, 
translator = Translator()
ge_comfort_features = df[['comfortfeatures::items::id', 'comfortfeatures::items::title']].dropna()
ge_comfort_features.columns = ["id", "ge"]
ge_comfort_features

In [ ]:
comfort_features = ge_comfort_features.copy()

# google denies too large/frequent requests

step = 10
max_size = 31
for i in range(0, max_size, step):
    comfort_features["ge"][i:min(i+step, max_size)] = ge_comfort_features["ge"][i:min(i+step, max_size)].apply(translator.translate, src="ka", dest="en").apply(getattr, args=('text',))
    time.sleep(1)

In [ ]:
comfort_features.rename(columns = {"ge": "en"}, inplace = True)
comfort_features

In [ ]:
comfort_features["id"] = comfort_features["id"].astype(int)
comfort_features

In [ ]:
for i in range(0, max_size):
    comf_id, comf_name = comfort_features.iloc[i, :]
    raw[comf_name.lower()] = raw["comfort_features"].apply(lambda x: False if type(x) is float else x.find(str(comf_id)) != -1)

In [ ]:
cols_to_drop = raw.nunique()
cols_to_drop = cols_to_drop.loc[cols_to_drop.values==1].index
raw.drop(columns = cols_to_drop, inplace = True)
raw.head()

# Train-Test split

In [ ]:
raw['order_date'] = pd.to_datetime(raw['order_date'], format="%Y-%m-%d %H:%M:%S")

raw.sort_values(by = ['order_date'], ascending = True, inplace = True)
train, test = train_test_split(raw, train_size = 0.6, shuffle = False)

raw.to_csv("../data/myauto-clean.csv")

In [ ]:
# no need to drop any tags in my split, all of them appear in the trainset
cols_to_drop = train.nunique()
cols_to_drop = cols_to_drop.loc[cols_to_drop.values==1].index
cols_to_drop

In [ ]:
train.to_csv("../data/train.csv", index = False)
test.to_csv("../data/test.csv", index = False)